# Labolatorium 5
Katarzyna Wcisło 


In [178]:
import pandas as pd
import numpy as np

tb = pd.read_csv('../OriginalData/tb.csv')
tb

,iso2,year,new_sp,new_sp_m04,new_sp_m514,new_sp_m014,new_sp_m1524,new_sp_m2534,new_sp_m3544,new_sp_m4554,...,new_sp_f04,new_sp_f514,new_sp_f014,new_sp_f1524,new_sp_f2534,new_sp_f3544,new_sp_f4554,new_sp_f5564,new_sp_f65,new_sp_fu
0,AD,1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AD,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AD,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AD,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AD,1993,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5764,ZW,2004,14581.0,NaN,NaN,187.0,833.0,2908.0,2298.0,1056.0,...,NaN,NaN,225.0,1140.0,2858.0,1565.0,622.0,214.0,111.0,NaN
5765,ZW,2005,13155.0,NaN,NaN,210.0,837.0,2264.0,1855.0,762.0,...,NaN,NaN,269.0,1136.0,2242.0,1255.0,578.0,193.0,603.0,NaN
5766,ZW,2006,12718.0,NaN,NaN,215.0,736.0,2391.0,1939.0,896.0,...,NaN,NaN,237.0,1020.0,2424.0,1355.0,632.0,230.0,96.0,NaN
5767,ZW,2007,10583.0,6.0,132.0,138.0,500.0,3693.0,0.0,716.0,...,7.0,178.0,185.0,739.0,3311.0,0.0,553.0,213.0,90.0,NaN


In [179]:
# Zakładam, że kolumna new_sp odpowieda sumie przypadków w każdym roku dla danego iso2. Tworzymy nową tabelę, 
# która nie zawiera wierszy, gdzie suma przypadków dla danego wieku i płci nie jest równa new_sp
tb_without_nan = tb.fillna(0.0)
tb_without_sp014 = tb_without_nan.drop(['new_sp_m014', 'new_sp_f014'],axis = 1)  #usuwam kolumnę dla wieku 0-14 ponieważ jest to suma kolumn 04 oraz 514
sum_sp = tb_without_nan.iloc[:,3:].sum(axis = 1)
tb_equal_sum = tb_without_sp014.drop(tb_without_sp014 [(tb_without_sp014 .new_sp != sum_sp)].index)
tb_equal_sum_no_zero = tb_equal_sum.drop(tb_equal_sum[(sum_sp == 0.0)].index)
tb_equal_sum_no_zero.to_csv('../AnalysisData/tb_without_uncorrect_cells.csv')



C:\Users\kasia\AppData\Local\Temp\ipykernel_13884\2043363059.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tb_equal_sum_no_zero = tb_equal_sum.drop(tb_equal_sum[(sum_sp == 0.0)].index)


In [180]:
tb_melt= tb_equal_sum.melt(id_vars=['iso2','year'], value_vars=tb_equal_sum.columns[3:])   
tb_melt.to_csv('../AnalysisData/tb_melt.csv')

tb_melt_without_zero = tb_melt.drop(tb_melt[(tb_melt.value == 0.0)].index)

Sex = ['Male' if str(row)[7:8]== 'm' else 'Female' for row in tb_melt_without_zero ['variable']]
tb_melt_without_zero['Sex'] = Sex

In [181]:
def Age(age: str):
    switch= {
        '04': '0-4',
        '514': '5-14',
        '1524':'15-24',
        '2534':'25-34',
        '3544':'35-44',
        '4564':'45-64',
        '65':'65',
        'u':'under 65'
    }
    return switch.get(age)
    
Age =[Age(str(row)[8:]) for row in tb_melt_without_zero['variable']]
tb_melt_without_zero['Age'] = Age

ending_table = tb_melt_without_zero.drop('variable', axis = 1)
new_column_order = ending_table.iloc[:,[0,1,3,4,2]]
new_column_name = {'iso2':'Country','year':'Year','value': 'Cases'}
new_tb  = new_column_order.rename(columns = new_column_name)
new_tb.to_csv('../AnalysisData/new_tb.csv')